In [138]:
#from gurobipy import *
import gurobipy as gp
import numpy as np
from gurobipy import GRB
import csv

def crear_red(path_datos,path_arcos):
    with open(path_datos) as datos:
        csv_datos = csv.reader(datos, delimiter=';')
        lista_datos = []
        for row in csv_datos:
            lista_datos.append(row)
    
    nodos = int(lista_datos[0][1])
    cantidad_arcos = int(lista_datos[1][1])
    nodo_inicio = int(lista_datos[2][1]) 
    nodo_fin = int(lista_datos[3][1])
     
    
    with open(path_arcos) as arcos:
        csv_arcos = csv.reader(arcos, delimiter=';')
        lista_arcos = []
        for row in csv_arcos:
            lista_arcos.append(row)

    lista_arcos = lista_arcos[1:]
    indice = 0
    dic_arco_indice = dict()
    dic_indice_capacidad = dict()
    
    #Creamos un diccionario donde se relaciona cada arco y su capacidad con un índice. 
    # Además se separan los arcos en listas para futuras restricciones.

    arcos_inicio = [[],[]]
    arcos_fin = [[],[]]
    arcos_centro = {}
    for j in range(0, nodos):
        if j != nodo_inicio and j != nodo_fin:
            arcos_centro[j] = [[],[]]

    for i in range(0, cantidad_arcos):
        linea = lista_arcos[i]
        arco = (int(linea[0]),int(linea[1]))
        dic_arco_indice[arco] = indice
        dic_indice_capacidad[indice] = float(linea[2])
        
        if int(linea[0]) == nodo_inicio:
            arcos_inicio[0].append(indice)
        
        elif int(linea[1]) == nodo_inicio:
            arcos_inicio[1].append(indice)
        
        elif int(linea[0]) == nodo_fin:
            arcos_fin[0].append(indice)
        
        elif int(linea[1]) == nodo_fin:
            arcos_fin[1].append(indice)
        
        else:
            arcos_centro[int(linea[0])][1].append(indice) #Arco sale de i
            arcos_centro[int(linea[1])][0].append(indice) #Arco llega a j
        
        indice += 1
    
    print(arcos_centro[2]) 
        
        
    
    return [nodo_inicio,nodo_fin,lista_arcos,dic_arco_indice, dic_indice_capacidad, arcos_inicio, arcos_fin, arcos_centro]

# La siguiente función recibe un vector "camino" que corresponde al vector X descrito en el informe. 
# Sus componentes toman valor 1 si es que este camino pasa por el arco con ese índice y
# 0 en otro caso.
# Entrega la suma de flujos de los arcos por los que pasa el camino, o sea el vector Vp del informe.

def capacidad_camino(camino, diccionario): 
    suma = 0
    for i in range(len(camino)):
        capacidad_arco = diccionario[i]
        suma += camino[i]*capacidad_arco

    return suma


In [139]:
def camino_inicial(nodo_inicio, nodo_termino, dic_arco_indice, lista_arcos):
    i = nodo_inicio
    nodos_visitados = []
    sin_salida = []
    indices_arcos = []
    camino_inicial = [0]*len(lista_arcos)
    j = 0
    
    x = 0
    while j != nodo_termino:
        siguiente = False
        x = 0

        while not siguiente and x<len(lista_arcos):
            factible = int(lista_arcos[x][1]) not in nodos_visitados and int(lista_arcos[x][1]) not in sin_salida
            if int(lista_arcos[x][0]) == i and factible:
                j = int(lista_arcos[x][1])
                tupla = (i,j)
                indice = dic_arco_indice[tupla]
                indices_arcos.append(indice)
                siguiente = True
                nodos_visitados.append(i)
                i = j
            else:
                x += 1
        
        if not siguiente:
            indices_arcos = indices_arcos[:-1]
            nodo_anterior = nodos_visitados.pop()
            sin_salida.append(i)
            i = nodo_anterior
      
    # Dado que si es que el camino pasa o no por un arco es una variable binaria, se transforman a 1 todos
    # los compenentes correspondientes del vector .
    for x in indices_arcos:
        camino_inicial[x] = 1

    return camino_inicial

In [140]:
path_datos = '/Users/clara/Desktop/Métodos de Opti/Tareas/Tarea 2/Datos1.csv'
path_arcos = '/Users/clara/Desktop/Métodos de Opti/Tareas/Tarea 2/Arcos1.csv'
   
red = crear_red(path_datos, path_arcos)
   
s = int(red[0])
t = int(red[1])
lista_arcos = red[2]
d_arco_indice = red[3]
d_indice_capacidad = red[4]

camino_inicial = camino_inicial(s,t,d_arco_indice,lista_arcos)
print(camino_inicial)

[[9, 20, 29, 41, 125], [23, 24, 25, 26, 27]]
[1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 

Función que ejecuta el algoritmo

MAESTRO
Variable:
fij = flujo que pasa por arco (i,j)
Parámetros:
xij = vector binario que cuyas filas que valen 1 corresponden a los índices de los arcos por los que pasa el camino.
P = matriz con todos los xij, cada columna corresponde a un camino.
uij = capacidad del arco (i,j)
U = vector de capacidades

SATÉLITE
Variable:
xij = vector binario que cuyas filas que valen 1 corresponden a los índices de los arcos por los que pasa el camino.
Parámetros:
fij = flujo que pasa por arco (i,j)
yij = variables duales óptimas del maestro


In [144]:
def resolver_problema_generacion_columnas(nodo_inicio, nodo_termino, camino_inicial, dic_cap, arcos_inicio, arcos_fin, arcos_centro, error):
    
    NITER = 10
    iteracion = 0
    K = 0
   

    cantidad_arcos = len(camino_inicial)
    caminos = [camino_inicial]
    capacidades = capacidad_camino(camino_inicial, dic_cap)
 
    # Problema Maestro
    maestro = gp.Model("Problema maestro")
    
    # Se crea un diccionario donde la llave es el subindice de la variable f, es decir, el arco
    # y el valor es la variable del flujo en ese arco
    f = []
    for k in range(len(caminos)-1):
        for i in range(cantidad_arcos):
            f[(k,i)] = maestro.addVar(lb=0, name= f'Flujo camino {k}, arco {i}')
    maestro.update()
    maestro.Params.OutputFlag = 0

    # Se crea un diccionario con las restricciones donde la llave es el subindice de
    # la restriccion de capacidad asociada a cada arco.
    capacidad = {}
    for i in range(cantidad_arcos):
        capacidad[i] = maestro.addConstr(sum(caminos[k][i]*f[(k,i)] for k in range(len(caminos)-1)) <= dic_cap[i], name=f'Cumplir cap. arco {i}')
        maestro.update()
    
    #Se crea la función objetivo del maestro
    maestro.setObjective(sum(sum(caminos[k][i]*f[(k,i)] for i in range(cantidad_arcos)) for k in range(0, len(caminos)-1)),GRB.MAXIMIZE)
    maestro.update()
    
    # Problema satélite
    # En él se va a usar la solución óptima del maestro
    
    x = {}
    for arco in range(cantidad_arcos):
        flujo = 0
        for camino in range(0, len(caminos)-1):
            flujo += f[(camino,arco)].X
            print(f[(camino,arco)].X)
        x[arco] = flujo
        
        
    satelite = gp.Model("Satelite")
    y = {}
    for i in range(0, cantidad_arcos):
        y[i] = satelite.addVar(lb = 0, ub = 1, name= f'Pasa por {i}')
    satelite.update()
    
    # Se agregan las restricciones de flujo
    F = sum(sum(caminos[k][i]*x[i] for i in range(0, cantidad_arcos-1)) for k in range(0, len(caminos)-1))
    
    # Flujo nodo inico (s)
    satelite.addConstr(sum(x[i] for i in arcos_inicio[0]) - sum(x[j] for j in arcos_inicio[1]) == F, name= 'Flujo inicio')

    # Flujos nodos centrales
    for k in arcos_centro:
       satelite.addConstr(sum(x[j] for j in k[1]) - sum(x[i] for i in k[0]) == F, name = f'Flujo central {k}' )
       
    # Flujo nodo fin (t)
    satelite.addConstr(sum(f[i] for i in arcos_fin[1]) - sum(f[j] for j in arcos_fin[0]) == -F, name= 'Flujo fin')
    
    satelite.update()
    
    #e agrega la función objetivo del satelite. Inicialmente se ponen índices ficticios.

    pi = [1]*cantidad_arcos
    satelite.setObjective(sum(y[i]*(x[i] - pi[i]) for i in range(cantidad_arcos)),GRB.MAXIMIZE)
    satelite.update()

    # Comienza el algoritmo
    while (iteracion < NITER) :
        # Se optimiza el problema maestro relajado
        iteracion = iteracion + 1
        print("\n\nIteración:", iteracion)
        maestro_relajado = maestro.relax()
        maestro_relajado.optimize()
        print("Objetivo del maestro relajado:", maestro_relajado.ObjVal)
    
        # Se almacenan las variables duales para luego ser ocupadas
        # en el subproblema y se define este problema.
        
        pi = [c.Pi for c in maestro_relajado.getConstrs()]

        # 3.5º Se resuelve el subproblema, se actualiza la función objetivo antes
        satelite.Params.OutputFlag = 0
        satelite.setObjective(sum(y[i]*(x[i] - pi[i]) for i in range(cantidad_arcos)),GRB.MAXIMIZE)
        satelite.update()
        satelite.optimize()

        print("Objetivo del satélite:", satelite.ObjVal)

        # Se termina el problema si se cumple el criterio de parada
        if satelite.ObjVal <= error: # break if no more columns
            break

        # En caso contrario, se generan un nuevo patron de corte y se 
        # agrega a la lista de patrones

        c = [int(y[i].X) for i in range(cantidad_arcos)]
        caminos.append(c)

        capacidad = capacidad_camino(c, dic_cap)
        print("Nueva capacidad", capacidad)

        
        # Se agrega la nueva columna al problema maestro
        # primero se define un objeto "columna" y se ponen ahí los 
        # datos del nuevo camino a agregar.
        col = gp.Column()
        for i in range(cantidad_arcos):
            col.addTerm(c[i])


        K = K + 1
        print("Fin de la iteración")
        
        
    print("Fin de la generacion de columnas")

    # Se resueve el maestro nuevamente, con todos los patrones generados
    # pero como problema entero
    # Primero hay que cambiar los tipos de las variables a "enteros"
    for var in maestro.getVars():
        var.vtype=GRB.INTEGER

    maestro.optimize()
    print("\n\n Valor objetivo final del problema:", maestro.ObjVal)
    


In [ ]:
error = 0.01
resolver_problema_generacion_columnas(s, t, camino_inicial, d_indice_capacidad, red[5],red[6], red[7], error)

1.c)

In [148]:
def resolver_p(camino_inicial, dic_cap, arcos_inicio, arcos_fin, arcos_centro, error):
    
    NITER = 10
    iteracion = 0
    K = 0

    cantidad_arcos = len(camino_inicial)
    caminos = [camino_inicial]
    capacidades = capacidad_camino(camino_inicial, dic_cap)
 
    # Problema Maestro
    p = gp.Model("Problema Original")
    
    # Se crea un diccionario donde la llave es el subindice de la variable f, es decir, el arco
    # y el valor es la variable del flujo en ese arco
    f = {}
    for k in range(cantidad_arcos):
        f[k] = p.addVar(lb=0, ub=capacidades[k], name= f'Flujo {k}')
        p.update()
    p.Params.OutputFlag = 0

     # Se agregan las restricciones de flujo
    F = sum(sum(caminos[k][i]*f[(k,i)] for i in range(0, cantidad_arcos-1)) for k in range(0, len(caminos)-1))
    
    # Flujo nodo inico (s)
    for k in caminos:
        p.addConstr(sum(f[(k,i)] for i in arcos_inicio[0]) - sum(f[(k,j)] for j in arcos_inicio[1]) == F, name= 'Flujo inicio')
        p.update()

    # Flujos nodos centrales
    for c in caminos:
        for k in arcos_centro:
            p.addConstr(sum(f[(c,j)] for j in k[1]) - sum(f[(c,i)] for i in k[0]) == F, name = f'Flujo central {k}' )
            p.update()
    # Flujo nodo fin (t)
    for k in caminos:
        p.addConstr(sum(f[(k,i)] for i in arcos_fin[1]) - sum(f[(k,j)] for j in arcos_fin[0]) == -F, name= 'Flujo fin')
        p.update()
    
    #Se crea la función objetivo del maestro
    p.setObjective(sum(sum(caminos[k][i]*f[(k,i)] for i in range(cantidad_arcos)) for k in range(0, len(caminos)-1)),GRB.MAXIMIZE)
    p.update()
    
    
    # Comienza el algoritmo
    while (iteracion < NITER) :
        # Se optimiza el problema 
        p.optimize()
        print("Objetivo:", p.ObjVal)
    
        if p.ObjVal <= error: # break if no more columns
            break

        K = K + 1
        print("Fin de la iteración")
        
        
    print("Fin de la optimización")
    
    


In [149]:
error = 0.01
resolver_p(camino_inicial, d_indice_capacidad, red[5],red[6], red[7], error)

TypeError: 'float' object is not subscriptable